In [27]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\MLPROJECT_part_prediction\\research'

In [26]:
os.chdir('e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\MLPROJECT_part_prediction\\research')

In [28]:
import os
os.chdir('../')

In [65]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\MLPROJECT_part_prediction'

In [82]:
#this command is used to set dags environment in jupyter notebook
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/hrishikeshbhagawati01/MLPROJECT_part_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="hrishikeshbhagawati01"
os.environ["MLFLOW_TRACKING_PASSWORD"]="54ea801af58531a14ca0eff221a0736561c9f604"

In [83]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [84]:
from src.ML_Part_predict.constants import *
from src.ML_Part_predict.utils.common import read_yaml,create_directories,save_json

class ConfigurationManager:
    def __init__(self,config_file_p=CONFIG_FILE_PATH,params_file_p=PARAMS_FILE_PATH,schema_file_p=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_p)
        self.params=read_yaml(params_file_p)
        self.schema=read_yaml(schema_file_p)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.random_forest  #
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])   

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/hrishikeshbhagawati01/MLPROJECT_part_prediction.mlflow"

        )

        return model_evaluation_config


In [85]:
import os
import pandas as pd
from sklearn.metrics import (accuracy_score,precision_score,recall_score)
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from src.ML_Part_predict.utils.common import save_json

class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config=config

    def eval_metrics(self,actual,pred):
        rec=recall_score(actual,pred)
        pre=precision_score(actual,pred)
        acc=accuracy_score(actual,pred)
        return rec,pre,acc
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rec,pre,acc) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"recall": rec, "precision": pre, "accuracy": acc}
            save_json(save_in_path=Path(self.config.metric_file_name), data=scores) ####
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("recall", rec)
            mlflow.log_metric("precision", pre)
            mlflow.log_metric("accuracy", acc)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForest")
            else:
                mlflow.sklearn.log_model(model, "model")


In [86]:
try:
    config=ConfigurationManager()
    model_evaluationconfig=config.get_model_evaluation_config()
    model_evaluationconfig=ModelEvaluation(config=model_evaluationconfig)
    model_evaluationconfig.log_into_mlflow()
except Exception as e:
    raise e

[2023-10-25 11:40:08,039: INFO: common: yaml file {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/hrishikesh147/MLProject/raw/main/part_namesz.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_dir': 'artifacts/data_ingestion/part_names.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_dir': 'artifacts/data_ingestion/part_names.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}, 'model_evaluation': {'root_dir': 'artifacts/model_evaluation', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_path': 'artifacts/model_trainer/model.

Registered model 'RandomForest' already exists. Creating a new version of this model...
2023/10/25 11:41:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForest, version 2
Created version '2' of model 'RandomForest'.
